In [1]:
import numpy as np

In [ ]:
# 実験：ニュートン型近接勾配法（Proximal Newton）を
# L1 正則化付き線形最小二乗（LASSO型）に適用する最小例
#
# 役割：
#  - Proximal_Newton  : 滑らか項 f を2次モデルで近似し、φ は prox で処理
#  - ProximalGradient_backtrack : サブ問題（2次モデル + φ）の近接勾配ソルバ
#  - soft_thresholding : φ(x)=C||x||_1 の近接写像（ソフト閾値）
from code8_6 import *  # ニュートン型近接勾配法 Proximal_Newton
from code8_2 import *  # 近接勾配法(バックトラッキング) ProximalGradient_backtrack
from code8_4 import *  # ソフト閾値関数 soft_thresholding

import numpy as np

# -------------------------------
# 問題データの生成（LASSO型）
# -------------------------------
n, m = 1000, 100  # 変数次元 n、サンプル数 m（m<n の疎推定シナリオ）
A = np.random.randn(m, n)  # デザイン行列 A ∈ R^{m×n}
b = np.random.randn(m)  # 観測ベクトル b ∈ R^{m}
C = 10  # L1 正則化の重み（φ(x)=C||x||_1）


# 滑らか項 f(x)=1/2||Ax-b||^2 とその勾配 ∇f(x)=A^T(Ax-b)
def LS(x):  # 目的関数 f(x)
    Axb = A @ x - b
    return (Axb @ Axb) / 2.0


def nab_LS(x):  # 勾配 ∇f(x)
    Axb = A @ x - b
    return A.T @ Axb


# 非滑らか項 φ(x)=C||x||_1（ProxNewton 本体には関数値が必要）
l1 = lambda x: C * np.linalg.norm(x, 1)

# 初期点（ゼロベクトル）
x_0 = np.zeros(n)

# -------------------------------
# ヘッセ行列（もしくはその近似）の指定
# -------------------------------
# f(x)=1/2||Ax-b||^2 の厳密ヘッセは ∇^2 f(x)=A^T A（x に依らず一定）
# Proximal_Newton は Hess(x) の形を取るため、呼び出し側で「x を受け取り A^T A を返す」
# ラッパを用意しておく（x には依存しない点に注意）。
AtA = lambda x: A.T @ A

# -------------------------------
# 解く（内部ではサブ問題を PGM-Backtracking で解き、Armijo 直線探索で更新）
# 収束しない／振動する場合は code8_6 側で：
#   - 直線探索係数 (tau, sig) の調整
#   - B_k（=A^T A）に微小正則化（B_k + δI）
# を検討する。
# -------------------------------
x_PN = Proximal_Newton(
    LS,  # f(x)
    nab_LS,  # ∇f(x)
    l1,  # φ(x)=C||x||_1
    soft_thresholding,  # prox_{αC}（ソフト閾値）
    C,  # 近接スケール C
    x_0,  # 初期点
    AtA,  # ヘッセ（ここでは一定行列 A^T A）
)

# （任意）最終目的値の確認
# print("Objective:", LS(x_PN) + l1(x_PN))